# Generators, Iterators, and Asynchronous Programming

## Tabla de contenidos
***


***

## Creando generadores

Los generadores fueron introducidos con la idea de introducing la iteración en Python, mientras se mejora el rendimiento del programa (usando menos memoria) al mismo tiempo.

La idea de un generador es crear un objeto que es iterable, y mientras es iterado, va a producir los elementos que contiene, uno a la vez. El uso principal de los generadores es guardar memoria, donde se tiene un objecto que sabe como producir cada elemento en particular, uno a la vez, cuando se es requerido.

## Un primer vistazo a los generadores

Empecemos con un ejemplo. El problema es que necesitamos procesar una lista larga de *records* y obtener algunas métricas e indicadores sobre ellos. Dado un dataset grande con información acerca de compras, queremos procesarlo para así obtener la venta más baja, la venta más alta y el precio promedio de venta. Para este ejemplo asumiremos que hay un archivo CSV con dos campos:
- `<purchase_date>, <price>`

Vamos a crear un objeto que recibe todas las compras, y nos entregará las métricas necesarias. El código que nos entregara los números es bastante simple. Es solo un objeto con un método que va a pocesar todos los precios de una, en cada step, va a actualizar el valor de cada métrica en particular en la cual estamos interesados.

In [1]:
class PurchasesStats:
    def __init__(self, purchases):
        self.purchases = iter(purchases)
        self.min_price: float = None
        self.max_price: float = None
        self._total_purchases_price: float = 0.0
        self._total_purchases = 0
        self.initialize()
        
    
    def _initialize(self):
        try:
            first_value = next(self.purchases)
        except StopIteration:
            raise ValueError("No values provided")
            
        
        self.min_price = self.max_price = first_value
        self._update_avg(first_value)
        
    
    def process(self):
        for purchase_value in self.purchases:
            self._update_min(purchase_value)
            self._update_max(purchase_value)
            self._update_avg(purchase_value)
        
        return self
    
    
    def _update_min(self, new_value: float):
        if new_value < self.min_price:
            self.min_price = new_value
            
    
    def _update_max(self, new_value: float):
        if new_value > self.max_price:
            self.max_price = new_value

    @property
    def avg_price(self):
        return self._total_purchases_price / self._total_purchases

    def _update_avg(self, new_value: float):
        self._total_purchases_price += new_value
        self._total_purchases += 1

    def __str__(self):
        return (
            f"{self.__class__.__name__}({self.min_price}, "
            f"{self.max_price}, {self.avg_price})"
        )

Ahora, se necesita una función que carga estos números en algo que este objeto pueda procesar. Una primera versión es

In [3]:
def _load_purchases(filename):
    purchases = []
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            purchases.append(float(price_raw))

    return purchases

Este código trabaja; carga todos los números del archivo en una lista, que cuando se le pasa a nuestro objecto, va a producir los números que queremos. Si se corre este código con un dataset muy grande, le va a tomar un cierto tiempo para completar e incluso podría fallar si el dataset es lo suficientemente largo hasta el punto de no caber en la memoria.

La solución es crear un generador. En vez de cargar todo el contenido del archivo en una lista, podemos producir los resultados uno a la vez.

In [4]:
def load_purchases(filename):
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            yield float(price_raw)

En este caso, la función `load_purchases` es una función generadora, o simplemente un generador.

En Python, la mera presencia de la palabra `yield` en cualquier función la convierte en un generador. Un objecto generador es iterable, lo que significa que puede funcionar con `for loops`.

Trabajar con iterables nos permite create este tipo de abstracciones poderosas que son polimórficas respecto a los `for loops`. Mientras queramos mantener la interfaz iterable, podemos iterar sobre este objeto de forma transparente.

## Generator expressions

Los generadores guardan mucha memoria, y dado que son iteradores, son una alternative conveniente a otros iterables o containers que requieren más espacio en memoria como listas, tuplas, o sets. También pueden ser definidos con *comprehensions*, solo que son llamados *generator expression*.

In [5]:
[x ** 2 for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [6]:
(x ** 2 for x in range(10))

<generator object <genexpr> at 0x000001D777B36F90>

In [7]:
sum(x ** 2 for x in range(10))

285

Mencionar que un generador va a estar "agotado" una vez que se ha iterado sobre el. Asegúrese que el resultado de la expresión es "consumido" una vez o va a obtener resultados inesperados.

## Iterating idiomatically

En esta ocasión queremos un objeto que produce una secuencia de números, desde uno, sin ningún límite. Un objeto como el siguiente puede lograr este objetivo. Cada vez que lo llamamos, obtenemos el siguiente número en la secuencia:

In [8]:
class NumberSequence:
    
    def __init__(self, start = 0):
        self.current = start
    
    
    def next(self):
        current = self.current
        self.current += 1
        return current

In [9]:
seq = NumberSequence()

In [10]:
seq.next()

0

In [11]:
seq.next()

1

Cada vez vamos a tener que usar el método `next()`. Notar que esta interface no soporta el ser iterada sobre un `for loop`, lo que significa que no lo podemos pasar como parámetro a funciones que esperan algo sobre lo que iterar.

In [12]:
list(zip(NumberSequence(), "abcdef"))

TypeError: 'NumberSequence' object is not iterable

El problema recae en que NumberSequence no soporta la iteración. Para arreglar esto, tenemos que hacer que el objeto sea iterable implementando el método mágico `__iter__()`. También hay que cambiar el método `next()` usando el método mágico `__next__`, que convierte al objeto en un iterador.

In [15]:
class SequenceOfNumbers:
    def __init__(self, start = 0):
        self.current = start
    
    
    def __next__(self):
        current = self.current
        self.current += 1
        return current
    
    def __iter__(self):
        return self

In [16]:
list(zip(SequenceOfNumbers(), "abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

Esto hace uso del protocolo de iteración. Este protocolo recae en los métodos `__iter__` y `__next__`.

## La función next()

La función de Python `next()` va a avanzar en el iterable a su siguiente elemento y lo va a retornar

In [17]:
word = iter("hello")

In [18]:
next(word)

'h'

In [19]:
next(word)

'e'

In [20]:
next(word)

'l'

In [21]:
next(word)

'l'

In [22]:
next(word)

'o'

In [23]:
next(word)

StopIteration: 

Si no se tienen más elementos para producir, se levantará la excepción `StopIteration`. La excepción señala que la iteración se ha terminado y que no hay más elementos para consumir.

Si deseamos manejar este caso, además de manejar la excepción `StopIteration`, le podríamos proveer a la función con un valor por defecto en su segundo parámetro. Si se le provee, va a retornar este valor cuando eventualmente se retornaría la excepción `StopIteration`

In [24]:
next(word, "default value")

'default value'

Es recomendable usar el valor por defecto la mayoría de las veces, para evitar tener excepciones en programas que están corriendo. La función `next()` puede ser muy útil en combinación con *generator expressions*, en situaciones en las que nos gustaría obtener los primeros elementos de un iterable que cumplen ciertos criterios.

## Usando un generador

El código previo puede ser significado notablemente usando un generador. De esta manera, en vez de crear una clase, podemos definir una función que *yields* los valores a medida que se necesitan:

In [25]:
def sequence(start = 0):
    while True:
        yield start
        start += 1

In [26]:
list(zip(sequence(), "abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

## Itertools